In [ ]:
#熱重啟
from keras.callbacks import Callback
import keras.backend as K
import numpy as np

class SGDRScheduler(Callback):

    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

In [ ]:
from keras.layers.normalization import BatchNormalization
import os
import json
import numpy as np
import pandas as pd
import keras
from keras import layers
from keras.applications import VGG19
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

INPUT_SIZE = 256

vgg19 = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(INPUT_SIZE,INPUT_SIZE,3)
)

model = keras.Sequential()
model.add(vgg19)
model.add(layers.GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))  #softmax
model.summary()

In [ ]:
# image preprocessing
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=False,
                                   channel_shift_range=10)

# featurewise_center=False,
# samplewise_center=False, 
# featurewise_std_normalization=False, 
# samplewise_std_normalization=False, 
# zca_whitening=False, 
# zca_epsilon=1e-06, 
# rotation_range=0, 
# width_shift_range=0.0, 
# height_shift_range=0.0, 
# brightness_range=None, 
# shear_range=0.0, 
# zoom_range=0.0, 
# channel_shift_range=0.0, 
# fill_mode='nearest', 
# cval=0.0, 
# horizontal_flip=False, 
# vertical_flip=False, 
# rescale=None, 
# preprocessing_function=None, 
# data_format=None, 
# validation_split=0.0, 
# dtype=None


valid_datagen = ImageDataGenerator(rescale=1./255)

INPUT_SIZE = 256
batch_size = 16
base_dir = "D:/OCR/id/test_flip"

training_set = train_datagen.flow_from_directory(base_dir+'/Train',
                                                 target_size=(INPUT_SIZE, INPUT_SIZE),
                                                 batch_size=batch_size,
                                                 class_mode='binary') # categorical

valid_set = valid_datagen.flow_from_directory(base_dir+'/Dev',
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            batch_size=batch_size,
                                            class_mode='binary')

train_num = training_set.samples
valid_num = valid_set.samples
print(train_num, valid_num)

In [ ]:
#熱重啟參數
schedule = SGDRScheduler(min_lr=1e-5,
                         max_lr=1e-2,
                         steps_per_epoch=np.ceil(train_num/batch_size),
                         lr_decay=0.9,
                         cycle_length=5,
                         mult_factor=1.5)

In [ ]:
from tensorflow.keras import optimizers
from keras.callbacks import EarlyStopping

model.compile(optimizer='SGD',
              loss='binary_crossentropy',  # categorical_crossentropy
              metrics=['acc']) # categorical_accuracy

# early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

train_num = training_set.samples
valid_num = valid_set.samples
history0 = model.fit_generator(training_set,
                    steps_per_epoch=train_num//batch_size,
                    validation_data=valid_set,
                    epochs=50,
                    validation_steps=valid_num//batch_size,
                    callbacks=[schedule, es])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history0.history['categorical_accuracy']
val_acc = history0.history['val_categorical_accuracy']
loss = history0.history['loss']
val_loss = history0.history['val_loss']
epochs = range(1, len(loss) + 1)

x=0
y=0
for i in val_acc[-(len(epochs)//5):]:
    x+=1
    y+=i
print(y/x)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()